In [1]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from data_helper import ImageGenerator, get_train_matrices, get_test_matrices
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
img_size = (224, 224)
output_size = 17

In [ ]:
resnet = ResNet50(include_top=True, weights='imagenet', input_shape=img_size + (3,))

In [ ]:
x_train, y_train, labels_map = get_train_matrices("../../planet-amazon-deforestation/input/train_v2.csv", "../../planet-amazon-deforestation/input/train-jpg", img_size)
X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)
generator = ImageGenerator()
flow = generator.get_train_generator(X_train, y_train)
validation_data = (X_valid, y_valid)

100%|██████████| 40479/40479 [05:27<00:00, 123.63it/s]


In [ ]:
x = resnet.output
# x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(output_size, activation='sigmoid')(x)
model = Model(input=resnet.input, output=predictions)

for layer in resnet.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy')

model.fit_generator(flow, samples_per_epoch=5, nb_epoch=10, validation_data=validation_data)